<a href="https://colab.research.google.com/github/mailynmailyn/CSI4142-Project/blob/main/Wildlife_Incidents_Database.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd

In [3]:
import numpy as np

In [4]:
# load datasets

incidents = pd.read_csv("incidents.csv", encoding = "cp1252")
staff = pd.read_csv("staff.csv", encoding = "cp1252")
responses = pd.read_csv("responses.csv", encoding = "cp1252")

# drop nulls in response db in Response Type Column
responses.drop(responses[responses["Response Type"].isnull()].index, inplace = True)

# combine Incident Date, Field Unit and Response Types for matching Incident Numbers 
aggregation_functions = {'Incident Date': 'first', 'Field Unit': 'first', 'Protected Heritage Area': 'first', 'Response Type': ', '.join}
responses = responses.groupby(responses['Incident Number']).aggregate(aggregation_functions)


In [55]:
# combine datasets by incident numbers 

# merge incidents, staff and responses databases. only include incidents from staff that are found in incidents table
merge_1 = pd.merge(incidents, staff, on= "Incident Number", how = "left")

db = pd.merge(merge_1, responses, on= "Incident Number", how = "left")

# check that repeated columns are matching (returning no unmatching values)
print(len(db.loc[(db['Incident Date_x'] != db['Incident Date_y']) | (db['Incident Date_x'] != db['Incident Date']) & (db['Incident Date'].isnull() == False)]))
print(len(db.loc[(db['Protected Heritage Area_x'] != db['Protected Heritage Area_y']) | (db['Protected Heritage Area_x'] != db['Protected Heritage Area']) & (db['Protected Heritage Area'].isnull() == False)]))
print(len(db.loc[(db['Field Unit_x'] != db['Field Unit_y']) | (db['Field Unit_x'] != db['Field Unit']) & (db['Field Unit'].isnull() == False)]))
print(len(db.loc[(db['Incident Type_x'] != db['Incident Type_y']) & (db['Incident Type_y'].isnull() == False)]))

# drop repeated and unimportant columns
db.drop(columns = ['Incident Date_x', 'Incident Date_y', 'Protected Heritage Area_x', 'Protected Heritage Area_y', 'Field Unit_x', 'Field Unit_y', 'Incident Type_y', 'Animal Response to Deterrents', 'Within Park'], inplace = True)

# rename Incident Type_x
db.rename(columns = {"Incident Type_x" : 'Incident Type'}, inplace=True)

# replace 'Unknown' values as empty
db.replace('Unknown', np.NaN, inplace = True)

# drop rows with 0 animals involved
db.drop(db[db["Sum of Number of Animals"] == 0].index, inplace = True)

# check for NaN values in animal num
print(len(db.loc[db['Sum of Number of Animals'].isnull()]))

# remove rows with 4 or more blanks
db["blank_count"] = db.apply(lambda x: x.count(), axis=1)

db.drop(db[db["blank_count"] < 14].index, inplace = True)

db

0
0
0
0
0


,Incident Number,Incident Type,Species Common Name,Sum of Number of Animals,Animal Health Status,Cause of Animal Health Status,Animal Behaviour,Reason for Animal Behaviour,Animal Attractant,Deterrents Used,Latitude Public,Longitude Public,Total Staff Involved,Total Staff Hours,Incident Date,Field Unit,Protected Heritage Area,Response Type,blank_count
0,BAN2010-0003,Human Wildlife Interaction,Coyote,2,Healthy,NaN,Avoidance,Surprise,Prey animal (natural),Presence of Officer/Person,51.161093,-115.593386,1.0,2.330000,2010-01-01,Banff Field Unit,Banff National Park of Canada,"Dispose Carcass, Investigate Incident, Monitor...",17
1,BAN2010-0003,Human Wildlife Interaction,Elk,1,Dead,Predation,NaN,NaN,NaN,NaN,51.161093,-115.593386,1.0,2.330000,2010-01-01,Banff Field Unit,Banff National Park of Canada,"Dispose Carcass, Investigate Incident, Monitor...",14
2,BAN2010-0003,Human Wildlife Interaction,Wolf,3,Not Located,NaN,NaN,NaN,Prey animal (natural),NaN,51.161093,-115.593386,1.0,2.330000,2010-01-01,Banff Field Unit,Banff National Park of Canada,"Dispose Carcass, Investigate Incident, Monitor...",14
3,JNP2010-0011,Rescued/Recovered/Found Wildlife,White-tailed Deer,1,Dead,Collision,NaN,NaN,NaN,NaN,53.139120,-117.964219,1.0,1.000000,2010-01-01,Jasper Field Unit,Jasper National Park of Canada,Dispose Carcass,14
5,JNP2010-0023,Rescued/Recovered/Found Wildlife,Mule Deer,1,Not Located,Collision,NaN,NaN,NaN,NaN,52.858415,-118.102814,1.0,3.000000,2010-01-01,Jasper Field Unit,Jasper National Park of Canada,Investigate Incident,14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73677,2021-HWC-0796-MRGFU-0060,Human Wildlife Interaction,Caribou,1,Healthy,NaN,Not Applicable,NaN,NaN,NaN,51.244278,-117.689360,2.0,11.000000,2021-12-24,Mount Revelstoke and Glacier Field Unit,Glacier National Park of Canada,Close Area,14
73678,2022-HWC-0594-WATFU-0001,Human Wildlife Interaction,Elk,1,Dead,Collision,NaN,NaN,NaN,NaN,49.120796,-113.820000,2.0,4.000000,2021-12-24,Waterton Lakes Field Unit,Waterton Lakes National Park of Canada,Leave on Landscape,14
73682,2021-HWC-0000-JASFU-2857,Human Wildlife Interaction,Elk,1,NaN,NaN,Presence - Wildlife Exclusion Zones,Habituation,NaN,Visual - Flagging or stick,52.872946,-118.094517,1.0,0.500000,2021-12-28,Jasper Field Unit,Jasper National Park of Canada,Haze - Soft,15
73684,2021-HWC-0000-JASFU-2858,Human Wildlife Interaction,Elk,1,NaN,NaN,Presence - Wildlife Exclusion Zones,Habituation,NaN,Non-impact - Chalkball,52.874856,-118.092955,1.0,0.333333,2021-12-29,Jasper Field Unit,Jasper National Park of Canada,Haze - Soft,15


In [79]:
import random
from string import digits

# Staff ID
db['Staff ID'] = ['S' + ''.join(random.choice(digits) for x in range(13)) for _ in range(len(db))]

# Animal Involvement ID
db['AI ID'] = ['AI' + ''.join(random.choice(digits) for x in range(13)) for _ in range(len(db))]

# Animal ID
db['Animal ID'] = ['A' + ''.join(random.choice(digits) for x in range(13)) for _ in range(len(db))]

# Location ID
db['Location ID'] = ['L' + ''.join(random.choice(digits) for x in range(13)) for _ in range(len(db))]

db.reset_index(drop=True, inplace = True)

# check for primary key duplicates
print(db.loc[db["Staff ID"].duplicated()])
print(db.loc[db["AI ID"].duplicated()])
print(db.loc[db["Animal ID"].duplicated()])
print(db.loc[db["Location ID"].duplicated()])


Empty DataFrame
Columns: [Incident Number, Incident Type, Species Common Name, Sum of Number of Animals, Animal Health Status, Cause of Animal Health Status, Animal Behaviour, Reason for Animal Behaviour, Animal Attractant, Deterrents Used, Latitude Public, Longitude Public, Total Staff Involved, Total Staff Hours, Incident Date, Field Unit, Protected Heritage Area, Response Type, blank_count, Staff ID, AI ID, Animal ID, Location ID, geometry]
Index: []

[0 rows x 24 columns]
Empty DataFrame
Columns: [Incident Number, Incident Type, Species Common Name, Sum of Number of Animals, Animal Health Status, Cause of Animal Health Status, Animal Behaviour, Reason for Animal Behaviour, Animal Attractant, Deterrents Used, Latitude Public, Longitude Public, Total Staff Involved, Total Staff Hours, Incident Date, Field Unit, Protected Heritage Area, Response Type, blank_count, Staff ID, AI ID, Animal ID, Location ID, geometry]
Index: []

[0 rows x 24 columns]
Empty DataFrame
Columns: [Incident Num

In [80]:
# incident table

incident_table = db[["Incident Number", "Incident Date", "Incident Type"]]
incident_table.drop_duplicates(subset=['Incident Number'], inplace = True)
incident_table.reset_index(drop=True, inplace= True)

incident_table.to_csv("Incident_Table.csv",  index=False)

incident_table

<ipython-input-80-4d2fc8dd6404>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  incident_table.drop_duplicates(subset=['Incident Number'], inplace = True)


,Incident Number,Incident Date,Incident Type
0,BAN2010-0003,2010-01-01,Human Wildlife Interaction
1,JNP2010-0011,2010-01-01,Rescued/Recovered/Found Wildlife
2,JNP2010-0023,2010-01-01,Rescued/Recovered/Found Wildlife
3,JNP2010-0016,2010-01-02,Rescued/Recovered/Found Wildlife
4,LL2010-000001,2010-01-02,Rescued/Recovered/Found Wildlife
...,...,...,...
41937,2021-HWC-0796-MRGFU-0060,2021-12-24,Human Wildlife Interaction
41938,2022-HWC-0594-WATFU-0001,2021-12-24,Human Wildlife Interaction
41939,2021-HWC-0000-JASFU-2857,2021-12-28,Human Wildlife Interaction
41940,2021-HWC-0000-JASFU-2858,2021-12-29,Human Wildlife Interaction


In [81]:
# animal table

animal_table = db[['Animal ID', 'Species Common Name', 'Animal Health Status', 'Sum of Number of Animals']]

animal_table.to_csv("Animal_Table.csv", index=False)

animal_table

,Animal ID,Species Common Name,Animal Health Status,Sum of Number of Animals
0,A8959677656270,Coyote,Healthy,2
1,A8564104582929,Elk,Dead,1
2,A4776419824418,Wolf,Not Located,3
3,A1306817214515,White-tailed Deer,Dead,1
4,A7776745761910,Mule Deer,Not Located,1
...,...,...,...,...
43972,A3124076556890,Caribou,Healthy,1
43973,A3719682661520,Elk,Dead,1
43974,A8255655455277,Elk,NaN,1
43975,A8257435481895,Elk,NaN,1


In [82]:
# animal involvement table

ai_table = db[['AI ID', 'Cause of Animal Health Status', 'Animal Behaviour','Reason for Animal Behaviour', 'Animal Attractant', 'Deterrents Used']]

ai_table.to_csv("Animal_Involvement_Table.csv", index=False)

ai_table


,AI ID,Cause of Animal Health Status,Animal Behaviour,Reason for Animal Behaviour,Animal Attractant,Deterrents Used
0,AI0011713945136,NaN,Avoidance,Surprise,Prey animal (natural),Presence of Officer/Person
1,AI9349066058609,Predation,NaN,NaN,NaN,NaN
2,AI8899821327959,NaN,NaN,NaN,Prey animal (natural),NaN
3,AI5534653879738,Collision,NaN,NaN,NaN,NaN
4,AI9384518257730,Collision,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
43972,AI4887690858436,NaN,Not Applicable,NaN,NaN,NaN
43973,AI0581133146974,Collision,NaN,NaN,NaN,NaN
43974,AI5508624572838,NaN,Presence - Wildlife Exclusion Zones,Habituation,NaN,Visual - Flagging or stick
43975,AI8406909591828,NaN,Presence - Wildlife Exclusion Zones,Habituation,NaN,Non-impact - Chalkball


In [25]:
import geopandas as gpd

In [83]:
# location table

# create point geometries from lon and lat
gdb = db

gdb.replace('Unknown', np.NaN, inplace = True)

gdb["geometry"] = gpd.points_from_xy(db['Longitude Public'], db['Latitude Public'], crs="EPSG:4326")

location_table = gdb[['Location ID', 'geometry', 'Protected Heritage Area']]

location_table.to_csv("Location_Table.csv", index=False)

location_table

,Location ID,geometry,Protected Heritage Area
0,L8231473405987,POINT (-115.59339 51.16109),Banff National Park of Canada
1,L7763977445564,POINT (-115.59339 51.16109),Banff National Park of Canada
2,L8599913299091,POINT (-115.59339 51.16109),Banff National Park of Canada
3,L1216398134474,POINT (-117.96422 53.13912),Jasper National Park of Canada
4,L7501478868603,POINT (-118.10281 52.85842),Jasper National Park of Canada
...,...,...,...
43972,L9813798783375,POINT (-117.68936 51.24428),Glacier National Park of Canada
43973,L3255115052980,POINT (-113.82000 49.12080),Waterton Lakes National Park of Canada
43974,L0779513705060,POINT (-118.09452 52.87295),Jasper National Park of Canada
43975,L5045226652893,POINT (-118.09296 52.87486),Jasper National Park of Canada


In [87]:
# staff

staff_table = db[['Staff ID', 'Field Unit', 'Total Staff Involved', 'Total Staff Hours']]

staff_table.to_csv("Staff_Table.csv",  index=False)

staff_table

,Staff ID,Field Unit,Total Staff Involved,Total Staff Hours
0,S6062127882632,Banff Field Unit,1.0,2.330000
1,S3703212399182,Banff Field Unit,1.0,2.330000
2,S5706416443647,Banff Field Unit,1.0,2.330000
3,S1755951152507,Jasper Field Unit,1.0,1.000000
4,S3242253843194,Jasper Field Unit,1.0,3.000000
...,...,...,...,...
43972,S6662149914485,Mount Revelstoke and Glacier Field Unit,2.0,11.000000
43973,S6712509242189,Waterton Lakes Field Unit,2.0,4.000000
43974,S7005801704766,Jasper Field Unit,1.0,0.500000
43975,S2959719941577,Jasper Field Unit,1.0,0.333333


In [85]:
# FACT TABLE 

fact_table = gdb[['Incident Number', 'AI ID', 'Animal ID', 'Staff ID', 'Location ID']]

fact_table.to_csv("Fact_Table.csv", index=False)

fact_table

,Incident Number,AI ID,Animal ID,Staff ID,Location ID
0,BAN2010-0003,AI0011713945136,A8959677656270,S6062127882632,L8231473405987
1,BAN2010-0003,AI9349066058609,A8564104582929,S3703212399182,L7763977445564
2,BAN2010-0003,AI8899821327959,A4776419824418,S5706416443647,L8599913299091
3,JNP2010-0011,AI5534653879738,A1306817214515,S1755951152507,L1216398134474
4,JNP2010-0023,AI9384518257730,A7776745761910,S3242253843194,L7501478868603
...,...,...,...,...,...
43972,2021-HWC-0796-MRGFU-0060,AI4887690858436,A3124076556890,S6662149914485,L9813798783375
43973,2022-HWC-0594-WATFU-0001,AI0581133146974,A3719682661520,S6712509242189,L3255115052980
43974,2021-HWC-0000-JASFU-2857,AI5508624572838,A8255655455277,S7005801704766,L0779513705060
43975,2021-HWC-0000-JASFU-2858,AI8406909591828,A8257435481895,S2959719941577,L5045226652893
